# Quick Start

Import the packages that we'll use:

In [1]:
import awkward as ak
import awkward_pandas as akpd
import numpy as np
import pandas as pd

Check what versions we have:

In [2]:
for entry in (ak, akpd, np, pd):
    print(f"{entry.__name__:15} {entry.__version__}")

awkward         2.0.0rc2
awkward_pandas  2022.8a1.dev14+gdf2fda7.d20221031
numpy           1.23.4
pandas          1.5.1


Make a simple awkward array:

In [3]:
a = ak.from_iter([[1, 2, 3], [4, 5], [6]] * 5)

In [4]:
a

<Array [[1, 2, 3], [4, 5], [6], ..., [4, 5], [6]] type='15 * var * int64'>

We get a series representation of the awkward array by using awkward-pandas `form_awkward` function:

In [5]:
s = akpd.from_awkward(a, name="a")

In [6]:
s

0     [1, 2, 3]
1        [4, 5]
2           [6]
3     [1, 2, 3]
4        [4, 5]
5           [6]
6     [1, 2, 3]
7        [4, 5]
8           [6]
9     [1, 2, 3]
10       [4, 5]
11          [6]
12    [1, 2, 3]
13       [4, 5]
14          [6]
Name: a, dtype: awkward

We can put the series in a DataFrame with another built-in pandas type, e.g. a column of integers:

In [7]:
df = pd.DataFrame({"integers": np.arange(42, 42 + len(s)), "awkwardstuff": s})

In [8]:
df

,integers,awkwardstuff
0,42,"[1, 2, 3]"
1,43,"[4, 5]"
2,44,[6]
3,45,"[1, 2, 3]"
4,46,"[4, 5]"
5,47,[6]
6,48,"[1, 2, 3]"
7,49,"[4, 5]"
8,50,[6]
9,51,"[1, 2, 3]"


With the DataFrame we can start doing usual pandas operations. Here we query the DataFrame based on the column of integers; selecting rows where the integer is even:

In [9]:
df.query("integers%2 == 0")

,integers,awkwardstuff
0,42,"[1, 2, 3]"
2,44,[6]
4,46,"[4, 5]"
6,48,"[1, 2, 3]"
8,50,[6]
10,52,"[4, 5]"
12,54,"[1, 2, 3]"
14,56,[6]


We can use DataFrame and Series methods:

In [10]:
df.max()

integers        56
awkwardstuff     6
dtype: int64

In [11]:
df.mean()

integers        49.0
awkwardstuff     3.5
dtype: float64

In [12]:
df.awkwardstuff.min()

1

To use functions from the `awkward` library, or to access the underlying awkward array directly, we use the `ak` accessor:

In [13]:
df.awkwardstuff.ak

Here we'll use the accessor to show two different paths that provide the same numerical result represented with different objects:

In [14]:
df.awkwardstuff.ak.min(axis=1)

0     1
1     4
2     6
3     1
4     4
5     6
6     1
7     4
8     6
9     1
10    4
11    6
12    1
13    4
14    6
dtype: awkward

In [15]:
ak.min(df.awkwardstuff.ak.array, axis=1)

<Array [1, 4, 6, 1, 4, 6, 1, 4, 6, 1, 4, 6, 1, 4, 6] type='15 * ?int64'>

In both cases we are calling the `ak.min` function with the argument `axis=1`. The difference:
1. In the first call we are using the accessor on the `pd.Series` and therefore we return a `pd.Series`.
2. In the second call we are accessing the underlying array directly, still via the `ak` accessor, but we then call the `ak.min` function directly, so an awkward `Array` object is returned.

The second path should be rare when reaching for awkward-pandas. The purpose of awkward-pandas is to plug awkward-arrays into Pandas like workflows. If you find yourself reaching for the second type of call, think about if you actually need Pandas in hte first place! You may be find just using awkward-array. Of course, there will be occasional reasons to need to reach down to the underlying array, which is why we provide that interface.